In [4]:
import numpy as np
import pandas as pd
import scipy as scp
import scipy.stats as ss
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import spsolve
from mpl_toolkits import mplot3d
from matplotlib import cm
import scipy.special as scsp
from scipy.integrate import quad
# from FMNM.Processes import OU_process
from scipy.interpolate import RegularGridInterpolator
import sympy as sp # マクローリン展開用

### 数式の整理
時点 $t$ における満期 $T$ の債券価格 $y(t, T)$ は以下で表現される．
$$
y(t) = - \frac{a(t, T) + b(t, T) \cdot x(t, T)}{T - t} 
$$
a(t, T) と b(t,T) は以下の常微分方程式に従う．
$$
\frac{\partial}{\partial t} a(t, T) = \rho_{0} + b(t, T)' \Sigma \phi - \frac{1}{2} b(t, T)' \Sigma \Sigma' b(t, T)
$$
$$
\frac{\partial}{\partial t} b(t, T) = \rho + (K - \Sigma \Phi) b(t, T)'
$$
ただし，リスクの市場価格 $\lambda(t)$ と短期金利 $r(t)$ は以下で表現される．
$$
\lambda(t) = \rho_{0} + \rho' x(t)
$$
$$
r(t) = \phi + \Phi x(t)
$$
/
ここで，リスクの市場価格 $\lambda(t)$ と短期金利 $r(t)$ に関連する $x(t)$ は以下の確率微分方程式に従う．
$$
\mathrm{d} x(t) = Kx(t) \mathrm{d}t + \Sigma \mathrm{d}B(t)
$$
分析に当たっては，以下の時系列モデルに従うとする．
$$
x(t) = e^{K(T - t)} x(t - 1)
$$
ただし，
$$
e^{K(T - t)} = \sum_{n=0}^{\infty} \frac{(K(T - t))^n}{n!}
$$
\
Kim and Wright においては，使用データは　(3か月, 6か月, 1年, 2年, ４年, 7年, 10年)　である．

### 最尤推定によるパラメータ推計対象
カルマンフィルタを用いるに当たり，最尤推定にて求めるべきパラメータは以下の通り．
$$
\rho_0 : (3 \times 1) \quad \mathrm{vector}
$$
$$
\rho : (3 \times 3) \quad \mathrm{vector}
$$
$$
\phi : (3 \times 1) \quad \mathrm{vector}
$$
$$
\Phi : (3 \times 3) \quad \mathrm{vector}
$$
$$
K : (3 \times 3) \quad \mathrm{vector}
$$
$$
\Sigma : (3 \times 3) \quad \mathrm{vector}
$$

In [52]:
# 使用データ (3か月, 6か月, 1年, 2年, ４年, 7年, 10年)
hyper_param = dict(
    max_maturity = 10, # 10年が最大値
    maturities_array = (3, 6, 12, 24, 48, 84, 120),
    approximate_order = 2, # 2次近似
    factor_num = 3,
    param_matrix_num = 4,
    param_vector_num = 2,
    initial_time_series_array = np.array([0, 0]) # ODEの初期値を設定
)
need_param_num = hyper_param["factor_num"] * hyper_param["param_vector_num"] + hyper_param["factor_num"] ** 2 * hyper_param["param_matrix_num"] 
initial_params = np.zeros(need_param_num)
need_param_num

42

In [6]:
def matrix_exponential(X, n):
    """
    行列 X の n 次のマクローリン展開を用いた指数関数の近似値を計算する。

    X       : 二次元の numpy 配列（行列）
    n       : マクローリン展開の次数
    
    return  : 行列の指数関数の近似値
    """
    result = np.zeros_like(X)
    power = np.eye(X.shape[0])
    
    for k in range(n + 1):
        result = result + power / np.math.factorial(k)
        power = np.dot(power, X)
    
    return result

In [47]:
def extract_parameters(params, factor_num, matrix_num, vector_num):
    """ パラメータ配列を適切な方に変換する関数
    
        params_list : list
            rho_0, phi, rho, K, Sigma
    """
    params_list = []
    for i in range(matrix_num + vector_num):
        if i <= vector_num - 1:
            print(factor_num * i, factor_num * (i + 1))
            params_list.append(np.array(params[factor_num * i : factor_num * (i + 1)]))
            end = factor_num * (i + 1) 
        else:
            start = end
            end = end + factor_num ** 2
            params_list.append(np.array(params[start : end]).reshape(factor_num, factor_num))
    return params_list

In [ ]:
def calc_state_var_value():
    """ 状態変数の時系列推移を計算する関数
    
    """

In [ ]:
def calc_yield(maturity, initial_time_series_array, params_list):
    """ イールドカーブを計算する関数
    
    """

    

In [ ]:
def kalman_function(yield_array, time, initial_state_vector, initial_covariance, params, maturities_array, approximate_order):
    """ カルマンフィルターのアルゴリズムを記載した関数
    
    """

    # パラメータを適切な形に変換
    param_list = extract_parameters(params, factor_num, param_matrix_num, param_vector_num)

    # 状態変数の初期値X_0 を設定
    state_vetor = initial_state_vector
    # 状態変数の分散の初期値Σ_0  を設定
    state_covariance = initial_covariance

    for maturity in maturities_array:
        
        # 金利の残差項 (実測値 - 推計値)
        resifural_array = yield_array[maturities_array] - 
